In [0]:
from __future__ import print_function
import easydict
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [0]:
class NetDrop(nn.Module):
  def __init__(self):
    super(NetDrop, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, 3, 1)
    self.conv2 = nn.Conv2d(32, 64, 3, 1)
    self.dropout1 = nn.Dropout2d(0.25)
    self.dropout2 = nn.Dropout2d(0.5)
    self.fc1 = nn.Linear(9216, 128)
    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = self.dropout1(x)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout2(x)
    output = self.fc2(x)
    return output

In [0]:
class NetConv(nn.Module):
  def __init__(self):
    super(NetConv, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, 3, 1)
    self.conv2 = nn.Conv2d(32, 64, 3, 1)
    self.fc1 = nn.Linear(36864, 128)
    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    output = self.fc2(x)
    return output

In [0]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(28*28, 500)
    self.fc2 = nn.Linear(500, 10)

  def forward(self, x):
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    output = self.fc2(x)
    return output

In [0]:
def train(args, model, device, train_loader, optimizer):
  model.train()
  # cross entropy loss
  criterion = nn.CrossEntropyLoss()
  for epoch in range(1, args.epochs + 1):
    for batch_idx, (image, target) in enumerate(train_loader):
      image, target = image.to(device), target.to(device)
      optimizer.zero_grad()
      
      output = model(image)
      
      loss = criterion(output, target)
      loss.backward()
      optimizer.step()
      
      if batch_idx % args.log_interval == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(image), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))
        

In [0]:
def test(args, model, device, test_loader):
  test_loss = 0
  correct = 0
  adv_correct = 0
  misclassified = 0
  criterion = nn.CrossEntropyLoss()
  for images, targets in test_loader:
    images = Variable(images.to(device), requires_grad=True)
    targets = Variable(targets.to(device))
   
    outputs = model(images)
    loss = criterion(outputs, targets)
    test_loss += loss
    loss.backward()

		# Generate perturbation
    grad = torch.sign(images.grad.data)
    adv_images = torch.clamp(images.data + args.epsilon * grad, 0, 1)

    adv_outputs = model(Variable(adv_images))

    _, pred = torch.max(outputs.data, 1)
    _, adv_preds = torch.max(adv_outputs.data, 1)

    correct += (pred == targets).sum().item()
    adv_correct += (adv_preds == targets).sum().item()
    misclassified += (pred != adv_preds).sum().item()


  test_loss /= len(test_loader.dataset)

  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f})\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))
  print('\nAdversarial Test: Accuracy: {}/{} ({:.0f})\n'.format(
      adv_correct, len(test_loader.dataset),
      100. * adv_correct / len(test_loader.dataset)))
  print('\nmisclassified examples : {}/ {}\n'.format(
      misclassified, len(test_loader.dataset)))

In [0]:
def main(network='fc', data_normalize=False):
  # args = parser.parse_args()
  args = easydict.EasyDict({
      'batch_size': 64,
      'test_batch_size': 1000,
      'epochs': 5,
      'lr': 0.001,
      'gamma': 0.7,
      'no_cuda': False,
      'log_interval': 64,
      'epsilon': 0.25
  })
  use_cuda = not args.no_cuda and torch.cuda.is_available()

  device = torch.device("cuda" if use_cuda else "cpu")
  
  transformation = transforms.ToTensor()
  if data_normalize:
    transformation = transforms.Compose([transforms.ToTensor(), 
                                         transforms.Normalize((0.1307,),(0.3081,))])  

  train_loader = torch.utils.data.DataLoader(
      datasets.MNIST('data/', train=True, download=True,
                      transform=transformation),
      batch_size=args.batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(
      datasets.MNIST('data/', train=False, download=True,
                     transform=transformation),
      batch_size=args.test_batch_size, shuffle=True)
  
  if network == 'conv':
    model = NetConv().to(device)
  elif network == 'drop':
    model = NetDrop().to(device)
  elif network == 'fc':
    model = Net().to(device)

  optimizer = optim.Adam(model.parameters(), lr= args.lr)

  train(args, model, device, train_loader, optimizer)

  test(args, model, device, test_loader)


In [15]:
# default - fc net
if __name__ == '__main__': 
  main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.311231
Train Epoch: 1 [4096/60000 (7%)]	Loss: 0.279794
Train Epoch: 1 [8192/60000 (14%)]	Loss: 0.489381
Train Epoch: 1 [12288/60000 (20%)]	Loss: 0.322116
Train Epoch: 1 [16384/60000 (27%)]	Loss: 0.249513
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.206595
Train Epoch: 1 [24576/60000 (41%)]	Loss: 0.283793
Train Epoch: 1 [28672/60000 (48%)]	Loss: 0.132506
Train Epoch: 1 [32768/60000 (55%)]	Loss: 0.350946
Train Epoch: 1 [36864/60000 (61%)]	Loss: 0.355547
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.200195
Train Epoch: 1 [45056/60000 (75%)]	Loss: 0.216296
Train Epoch: 1 [49152/60000 (82%)]	Loss: 0.092850
Train Epoch: 1 [53248/60000 (89%)]	Loss: 0.071636
Train Epoch: 1 [57344/60000 (96%)]	Loss: 0.125348
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.117366
Train Epoch: 2 [4096/60000 (7%)]	Loss: 0.066372
Train Epoch: 2 [8192/60000 (14%)]	Loss: 0.100264
Train Epoch: 2 [12288/60000 (20%)]	Loss: 0.102073
Train Epoch: 2 [16384/60000 (27%)]	Loss: 0.234614
Train Epoch: 2 [

In [16]:
# use convolutioal network model
if __name__ == '__main__':
  main(network='conv')

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.305729
Train Epoch: 1 [4096/60000 (7%)]	Loss: 0.380430
Train Epoch: 1 [8192/60000 (14%)]	Loss: 0.171780
Train Epoch: 1 [12288/60000 (20%)]	Loss: 0.103520
Train Epoch: 1 [16384/60000 (27%)]	Loss: 0.279369
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.110197
Train Epoch: 1 [24576/60000 (41%)]	Loss: 0.032505
Train Epoch: 1 [28672/60000 (48%)]	Loss: 0.075675
Train Epoch: 1 [32768/60000 (55%)]	Loss: 0.186955
Train Epoch: 1 [36864/60000 (61%)]	Loss: 0.007782
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.051641
Train Epoch: 1 [45056/60000 (75%)]	Loss: 0.015174
Train Epoch: 1 [49152/60000 (82%)]	Loss: 0.066235
Train Epoch: 1 [53248/60000 (89%)]	Loss: 0.172327
Train Epoch: 1 [57344/60000 (96%)]	Loss: 0.007114
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.043671
Train Epoch: 2 [4096/60000 (7%)]	Loss: 0.006706
Train Epoch: 2 [8192/60000 (14%)]	Loss: 0.012339
Train Epoch: 2 [12288/60000 (20%)]	Loss: 0.005360
Train Epoch: 2 [16384/60000 (27%)]	Loss: 0.007608
Train Epoch: 2 [

In [17]:
# use dropout network model
if __name__ == '__main__':
  main(network='drop')


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307730
Train Epoch: 1 [4096/60000 (7%)]	Loss: 0.475542
Train Epoch: 1 [8192/60000 (14%)]	Loss: 0.290257
Train Epoch: 1 [12288/60000 (20%)]	Loss: 0.190670
Train Epoch: 1 [16384/60000 (27%)]	Loss: 0.249261
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.328419
Train Epoch: 1 [24576/60000 (41%)]	Loss: 0.100960
Train Epoch: 1 [28672/60000 (48%)]	Loss: 0.098544
Train Epoch: 1 [32768/60000 (55%)]	Loss: 0.088276
Train Epoch: 1 [36864/60000 (61%)]	Loss: 0.104067
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.110589
Train Epoch: 1 [45056/60000 (75%)]	Loss: 0.040688
Train Epoch: 1 [49152/60000 (82%)]	Loss: 0.136981
Train Epoch: 1 [53248/60000 (89%)]	Loss: 0.064971
Train Epoch: 1 [57344/60000 (96%)]	Loss: 0.052227
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.137582
Train Epoch: 2 [4096/60000 (7%)]	Loss: 0.023206
Train Epoch: 2 [8192/60000 (14%)]	Loss: 0.096942
Train Epoch: 2 [12288/60000 (20%)]	Loss: 0.273607
Train Epoch: 2 [16384/60000 (27%)]	Loss: 0.041753
Train Epoch: 2 [

In [18]:
# data normalizationL True
if __name__ == '__main__':
  main(data_normalize=True)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303055
Train Epoch: 1 [4096/60000 (7%)]	Loss: 0.248802
Train Epoch: 1 [8192/60000 (14%)]	Loss: 0.252149
Train Epoch: 1 [12288/60000 (20%)]	Loss: 0.186621
Train Epoch: 1 [16384/60000 (27%)]	Loss: 0.246582
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.278320
Train Epoch: 1 [24576/60000 (41%)]	Loss: 0.206446
Train Epoch: 1 [28672/60000 (48%)]	Loss: 0.158319
Train Epoch: 1 [32768/60000 (55%)]	Loss: 0.183013
Train Epoch: 1 [36864/60000 (61%)]	Loss: 0.114978
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.037682
Train Epoch: 1 [45056/60000 (75%)]	Loss: 0.044962
Train Epoch: 1 [49152/60000 (82%)]	Loss: 0.073879
Train Epoch: 1 [53248/60000 (89%)]	Loss: 0.088708
Train Epoch: 1 [57344/60000 (96%)]	Loss: 0.490714
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.166194
Train Epoch: 2 [4096/60000 (7%)]	Loss: 0.034445
Train Epoch: 2 [8192/60000 (14%)]	Loss: 0.170235
Train Epoch: 2 [12288/60000 (20%)]	Loss: 0.107106
Train Epoch: 2 [16384/60000 (27%)]	Loss: 0.063629
Train Epoch: 2 [